# Get Historical S&P500 Members

In this file, we find months where not all historical S&P500 members are available. We then use Wikipedia's list of date-by-date additions and removals of companies to fill in the gaps.

## Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
input_dir = 'inputs/'
output_dir = 'outputs/'

sp500_members = pd.read_csv(input_dir + 'hist_sp500_members.csv')
sp500_members = sp500_members.drop_duplicates(subset=['Date', 'Ticker'], keep='first')
sp500_members['Date'] = pd.to_datetime(sp500_members['Date'])

bbg_all = pd.read_csv(input_dir + 'Bloomberg_Joined_Final.csv')
bbg_all.rename(columns={'Dates': 'Date'}, inplace=True)
bbg_all['Date'] = pd.to_datetime(bbg_all['Date'])

## Find Missing Months

In [3]:
# Find dates where historical data x BBG is missing
bbg_historical = sp500_members.merge(
    bbg_all, how='left', on=['Ticker', 'Date'], validate='1:1', indicator=True)
bbg_historical.value_counts('Date', ascending=True)

Date
2016-02-28      1
2016-09-30      1
2017-03-31      1
2013-01-31    500
2014-03-31    500
             ... 
2017-10-31    505
2017-01-31    505
2019-12-31    505
2016-10-31    506
2015-11-30    506
Length: 84, dtype: int64

In [4]:
# Get all changes in the error months where value_counts < 500
dates = bbg_historical.value_counts('Date', ascending=True)
dates = dates[dates < 500]
dates.index

DatetimeIndex(['2016-02-28', '2016-09-30', '2017-03-31'], dtype='datetime64[ns]', name='Date', freq=None)

## Get Historical Flows In and Out of S&P500

Source: https://analyzingalpha.com/sp500-historical-components-and-changes

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data = pd.read_html(url)

In [6]:
# Get the adjustments dataframe and rename columns
adjustments = data[1]
columns = ['date', 'ticker_added','name_added', 'ticker_removed', 'name_removed', 'reason']
adjustments.columns = columns

# Create additions dataframe.
additions = adjustments[~adjustments['ticker_added'].isnull()][['date','ticker_added', 'name_added']]
additions.columns = ['date','ticker','name']
additions['action'] = 'added'

# Create removals dataframe.
removals = adjustments[~adjustments['ticker_removed'].isnull()][['date','ticker_removed','name_removed']]
removals.columns = ['date','ticker','name']
removals['action'] = 'removed'

# Merge the additions and removals into one dataframe.
historical = pd.concat([additions, removals])

# Convert dates
historical['date'] = pd.to_datetime(historical['date'])
historical

date ticker                      name   action
0   2023-03-20   FICO                Fair Isaac    added
1   2023-03-15     BG             Bunge Limited    added
2   2023-03-15   PODD                   Insulet    added
4   2023-01-04   GEHC             GE HealthCare    added
5   2022-12-22   STLD            Steel Dynamics    added
..         ...    ...                       ...      ...
318 1999-06-09    HPH  Harnischfeger Industries  removed
319 1998-12-11    LDW                     Amoco  removed
320 1998-12-11    GRN                General Re  removed
321 1998-12-11    SUN                SunAmerica  removed
322 1997-06-17    USL                    USLife  removed

[623 rows x 4 columns]

## Update Dataframe Given New Data

In [7]:
# For each date in dates.index, get start date for that month
dates = pd.DataFrame(
    dates.index, columns=['Date']).rename(columns={'Date': 'End Date'})
dates['Start Date'] = pd.to_datetime(
    dates['End Date'].dt.year.astype(str) + '-' + \
    dates['End Date'].dt.month.astype(str) + \
    '-01')

# For each start and end date in dates, get the changes in the sp500_history
changes = []
for i in range(len(dates)):
  start_date = dates.iloc[i]['Start Date']
  end_date = dates.iloc[i]['End Date']
  curr_changes = historical.query('date >= @start_date and date <= @end_date')
  # remove any duplicates, including only the last instance of a ticker for sorted month
  curr_changes = curr_changes.drop_duplicates(subset=['ticker'], keep='last')
  # Get list of all added in action == added
  added = curr_changes[curr_changes['action'] == 'added']['ticker'].tolist()
  # Get list of all removed in action == removed
  removed = curr_changes[curr_changes['action'] == 'removed']['ticker'].tolist()
  changes.append([start_date, end_date, added, removed])
changes

[[Timestamp('2016-02-01 00:00:00'),
  Timestamp('2016-02-28 00:00:00'),
  ['CXO', 'CFG', 'FRT'],
  ['PCL', 'PCP', 'BRCM']],
 [Timestamp('2016-09-01 00:00:00'),
  Timestamp('2016-09-30 00:00:00'),
  ['COTY', 'COO', 'CHTR', 'MTD'],
  ['DO', 'HOT', 'EMC', 'TYC']],
 [Timestamp('2017-03-01 00:00:00'),
  Timestamp('2017-03-31 00:00:00'),
  ['AMD', 'RJF', 'ARE', 'SNPS', 'DISH', 'REG', 'CBOE'],
  ['URBN', 'FTR', 'FSLR', 'HAR', 'LLTC', 'ENDP', 'PBI']]]

In [8]:
# For missing months, copy the previous month, add added, remove removed
sp500_members['Simple Ticker'] = sp500_members['Ticker'].str.split(
    ' ', expand=True)[0]
for i in range(len(changes)):
  # Duplicate rows from previous month to current month
  prev_month = sp500_members.loc[
      sp500_members['Date'] == changes[i][0] - pd.DateOffset(days=1)]
  curr_month = prev_month.copy()
  curr_month['Date'] = changes[i][1]

  # Add tickers in added list
  to_add = pd.DataFrame({
      'Date': changes[i][0],
      'Ticker': changes[i][2],
      'Simple Ticker': changes[i][2]})
  curr_month = pd.concat((curr_month, to_add))

  # Remove tickers in removed list
  curr_month = curr_month[~curr_month['Simple Ticker'].isin(changes[i][3])]

  # Remove any tickers from curr_month that are already in historical_sp500
  curr_month = curr_month[~curr_month['Simple Ticker'].isin(
      sp500_members.loc[sp500_members['Date'] == changes[i][0]]['Simple Ticker'])]
  
  # Add curr_month to historical_sp500
  sp500_members = pd.concat((sp500_members, curr_month))
  # Print difference in tickers between previous month and current month
  print(prev_month[['Simple Ticker']].merge(curr_month[['Simple Ticker']], how='outer', on=['Simple Ticker'], indicator=True).value_counts('_merge', ascending=True))
  
sp500_members = sp500_members.sort_values(['Date', 'Ticker'])
sp500_members.drop(columns=['Simple Ticker'], inplace=True)

_merge
left_only       3
right_only      3
both          501
dtype: int64
_merge
left_only       2
right_only      4
both          503
dtype: int64
_merge
left_only       5
right_only      7
both          500
dtype: int64


In [9]:
sp500_members.to_csv(output_dir + 'hist_sp500_members.csv', index=False)